In [182]:
import pandas as pd
import numpy as np

In [183]:
lp  = pd.read_csv(open('all_loan_proposals.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)

In [184]:
lp['TransactionID'] = 0
lp['TransactionID'] = lp['PA'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + lp['PA'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + lp['ELA'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
lp['TransactionID'][:5]

0    2200G
1    2849P
2    7831B
3    1042G
4    0105C
Name: TransactionID, dtype: object

In [185]:
len(lp)

585

In [186]:
closings  = pd.read_csv(open('Closings_Pipeline/ClosingsData.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)#, index_col = 'TransactionID')
closings['Loan Amount'] = closings['Loan Amount'].astype(str)

In [187]:
closings['TransactionID'] = 0
closings['TransactionID'] = closings['Property Address'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + closings['Property Address'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + closings['Loan Amount'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
closings['TransactionID'][:5]

0    2484S
1    2643C
2    3644C
3    89023
4    0179Q
Name: TransactionID, dtype: object

In [188]:
len(closings)

1202

In [189]:
merged = pd.merge(closings, lp, how = 'left', on = 'TransactionID')
len(merged.columns)

63

In [190]:
#merged = merged.dropna(thresh=(len(merged.columns)-50), axis = 0)
#merged = merged.drop_duplicates('Property Address', keep='first')##
len(merged)

1206

In [191]:
merged.to_csv('all_lp_transactions.csv')

In [192]:
inactive_lps = pd.read_csv(open('inactive_loan_proposals.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)#, index_col = 'TransactionID')

In [193]:
merged = merged.append(inactive_lps, ignore_index = True)
len(merged)

1433

In [194]:
#merged = merged[merged['PA'].notnull()].drop_duplicates('PA', keep='first')##
len(merged)

1433

In [195]:
merged.to_csv('all_lp_transactions.csv') #Need to impute values and extract information about borrower from investor experience field

In [196]:
#clean IREIE column: eliminate + signs (for cases like 50+ flips) and replace number strings with ints
def clean_unstructured_data(dataframe, dirty_column, dirty_characters):
    for dirty_character in dirty_characters:
       dataframe[dirty_column] = dataframe[dirty_column].str.replace(dirty_character, '')
    return dataframe[dirty_column]
dirty_characters = ['+', '/', ',']
merged['IREIE'] = clean_unstructured_data(merged, 'IREIE', dirty_characters)
merged['IREIE'][:5]

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: IREIE, dtype: object

In [197]:
import re
#Cases for when borrower has completed one property
def first_property(x):
    keywords = ['property', 'completed', 'flip', 'rental', 'investment property', 'tenant occu', 'occup', 
                'rehab property', 'projects', 'prop', 'one', 'first property', 'first project', 'first rehab', 'first rental', 
                'home']
    pattern = '|'.join(keywords)
    first_property = re.search(pattern,x['IREIE'], flags=re.IGNORECASE)
    if(first_property):
            return '1'
    return '0'
merged['Completed Properties'] = merged[~merged['IREIE'].isnull()].apply(first_property, axis = 1)
merged[['Completed Properties','IREIE']][:20]


,Completed Properties,IREIE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [198]:

def properties_completed(x):
    keywords = ['properties', 'completed', 'flips', 'rental', 'investment', 'tenant occu', 'occup', 'rehab', 'projects', 'prop',
                'homes']
    for keyword in keywords:
        pattern = '\d+ (?='+keyword+')'
        m = re.search(pattern, x['IREIE'])
        if(m and m.group(0).rstrip().isdigit()):
            return m.group(0).rstrip()
    return x['Completed Properties']
merged['Completed Properties'] = merged[~merged['IREIE'].isnull()].apply(properties_completed, axis = 1)
merged[['Completed Properties','IREIE']][:20]

,Completed Properties,IREIE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [199]:
#Cases for when borrower has one year of experience
def first_year(x):
    keywords = ['year', 'yr', 'Yes', 'experience']
    pattern = '|'.join(keywords)
    first_year = re.search(pattern,x['IREIE'], flags=re.IGNORECASE)
    if(first_year):
            return '1'
    return '0'
merged['Years of Experience'] = merged[~merged['IREIE'].isnull()].apply(first_year, axis = 1)
merged[['Years of Experience','IREIE']][:20]

,Years of Experience,IREIE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [200]:
def years_experience(x):
    keywords = ['years', 'yrs']
    for keyword in keywords:
        pattern = '\d+ (?='+keyword+')'
        m = re.search(pattern, x['IREIE'])
        if(m and m.group(0).rstrip().isdigit()):
            return m.group(0).rstrip()
    return x['Years of Experience']
merged['Years of Experience'] = merged[~merged['IREIE'].isnull()].apply(years_experience, axis = 1)
merged[['Years of Experience','IREIE']][:5]

,Years of Experience,IREIE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [201]:
def is_repeat_borrower(x):
    keywords = ['repeat', 'BridgeWell Capital', 'BWC', 'performing', 'loan']
    pattern = '|'.join(keywords)
    is_repeat = re.search(pattern,x['IREIE'], flags=re.IGNORECASE)
    if(is_repeat):
            return '1'
    return '0'
merged['Repeat Borrower'] = merged[~merged['IREIE'].isnull()].apply(is_repeat_borrower, axis = 1)
merged[['Repeat Borrower','IREIE']][:5]

,Repeat Borrower,IREIE
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [202]:
merged.to_csv('all_lp_transactions.csv') #Need to impute values and extract information about borrower from investor experience field

In [203]:
len(merged)

1433

In [204]:
ltds  = pd.read_csv(open('all_ltds.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)
ltds.columns.values

array(['Unnamed: 0', u'Account Exec Name', u'BorrowerCompany Name',
       u'City', u'City_2', u'Co', u'Date', u'Down Payment', u'Email',
       u'Est Annual HOA Dues', u'Est Annual Ins',
       u'Est Annual Property Taxes', u'Est Close Costs and PrePaids',
       u'Est Closing Date', u'Est Lender title Co Closing Costs',
       u'Est Loan Maturity Date', u'Est PrePaid Days of Interest',
       u'Est PrePaid Days of Interest_2',
       u'Est Prepaid Home Owners Association Dues',
       u'Est Prepaid Property Insurance', u'Estimated Monthly Ins Escrow',
       u'Estimated Monthly Tax Escrow', u'Flood Insurance',
       u'Interest Rate', u'Interest Rate_2', u'Investor Name', u'Lien',
       u'Lien Loan', u'Lien Loan Amt', u'Lien Monthly Interest Payment',
       u'Lien Monthly Interest Payment_2', u'Lien_2', u'Lien_3', u'Lien_4',
       u'Loan Orig Fee', u'Loan Orig Fee_2',
       u'Loan Orig including RCL Orig Fee', u'Monthly Payment Date',
       u'Mos', u'Mos_2', u'Phone', u'Prepaid 

In [205]:
ltds['TransactionID'] = 0
ltds['TransactionID'] = ltds['Prop Add'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + ltds['Prop Add'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + ltds['Lien Loan Amt'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
ltds['TransactionID'][:5]

0    0001L
1    0100H
2    0101S
3    1014P
4    0104R
Name: TransactionID, dtype: object

In [206]:
merged = pd.merge(merged, ltds, how = 'left', on = 'TransactionID')
#merged.to_csv('all_transactions.csv')
len(merged)

1634

In [207]:
merged.to_csv('all_transactions.csv')

In [208]:
lqs  = pd.read_csv(open('all_lqs.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)
len(lqs)

661

In [209]:
lqs['TransactionID'] = 0
lqs['TransactionID'] = lqs[lqs['porpAddress'].notnull()]['porpAddress'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + lqs['porpAddress'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + lqs['Lien Loan Amt'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
lqs['TransactionID'][:5]

0    0154W
1    0101E
2    1027I
3      NaN
4    1042G
Name: TransactionID, dtype: object

In [210]:
#lqs['TransactionID']=lqs[lqs['Text12'].notnull()]['Text12'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
lqs['TransactionID2'] = 0
lqs['TransactionID2'] = lqs[lqs['Text12'].notnull()]['Text12'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + lqs['Text12'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + lqs['Lien Loan Amt'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
lqs['TransactionID2'][:5]

0      NaN
1      NaN
2      NaN
3    0104R
4      NaN
Name: TransactionID2, dtype: object

In [211]:
lqs.loc[lqs['TransactionID'].isnull(), 'TransactionID'] = lqs.loc[lqs['TransactionID'].isnull(), 'TransactionID2']
len(lqs[lqs['TransactionID'].notnull()]['TransactionID'])

656

In [212]:
merged = pd.merge(merged, lqs, how = 'left', on = 'TransactionID')
#merged.to_csv('all_transactions.csv')
len(merged)

3004

In [213]:
merged.to_csv('all_transactions.csv')

In [214]:
a_lqs  = pd.read_csv(open('all_a_lqs.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)

In [215]:
a_lqs['TransactionID'] = 0
a_lqs['TransactionID'] = a_lqs['Property Address'].str.split(' ').apply(pd.Series, 1)[0].str[:4].str.zfill(4) + a_lqs['Property Address'].str.split(' ').apply(pd.Series, 1)[1].str[:1]# + lqs['Lien Loan Amt'].str.split('.').apply(pd.Series, 1)[0].str[:4].str.zfill(4)
#a_lqs['TransactionID'][:5]
len(a_lqs)

775

In [216]:
merged = pd.merge(merged, a_lqs, how = 'left', on = 'TransactionID')
len(merged)

9818

In [217]:
merged.to_csv('all_transactions.csv')

In [218]:
merged.loc[merged['Closed'].isnull(), 'Closed'] = 0
len(merged[merged['Closed'].isnull()]['Closed'])

0

In [219]:
merged.to_csv('all_transactions.csv')

In [220]:
merged = merged.drop_duplicates('TransactionID', keep='first')

In [221]:
len(merged)

1162

In [222]:
merged.loc[merged['Property Address_x'].isnull(), 'Property Address_x'] = merged.loc[merged['Property Address_x'].isnull(), 'porpAddress']
merged.loc[merged['Property Address_x'].isnull(), 'Property Address_x'] = merged.loc[merged['Property Address_x'].isnull(), 'PA']
merged.loc[merged['Property Address_x'].isnull(), 'Property Address_x'] = merged.loc[merged['Property Address_x'].isnull(), 'Property Address_y']
len(merged[merged['Property Address_x'].isnull()]['Property Address_x'])

0

In [223]:
merged.to_csv('all_transactions.csv')
len(merged)

1162

In [224]:
len(merged[merged['cashReserves'].isnull()])

915

In [225]:
merged.loc[merged['cashReserves'].isnull(), 'cashReserves'] = merged.loc[merged['cashReserves'].isnull(), 'ICR']
len(merged[merged['cashReserves'].isnull()])

597

In [226]:
merged.loc[merged['cashReserves'].isnull(), 'cashReserves'] = merged.loc[merged['cashReserves'].isnull(), 'Cash Reserves (verifiable)']
len(merged[merged['cashReserves'].isnull()])

586

In [227]:
merged.loc[merged['cashReserves'].isnull(), 'cashReserves'] = merged.loc[merged['cashReserves'].isnull(), 'Text61']
len(merged[merged['cashReserves'].notnull()])

602

In [228]:
len(merged[merged['PP'].isnull()])

757

In [229]:
merged.loc[merged['PP'].isnull(), 'PP'] = merged.loc[merged['PP'].isnull(), 'Subject Property Contract Sales Price']
len(merged[merged['PP'].isnull()])

551

In [230]:
merged.loc[merged['PP'].isnull(), 'PP'] = merged.loc[merged['PP'].isnull(), 'Contract Purchase Price']
len(merged[merged['PP'].isnull()])

544

In [231]:
merged.loc[merged['PP'].isnull(), 'PP'] = merged.loc[merged['PP'].isnull(), 'purchasePrice']
len(merged[merged['PP'].isnull()])

498

In [232]:
merged.loc[merged['PP'].isnull(), 'PP'] = merged.loc[merged['PP'].isnull(), 'Text21']
len(merged[merged['PP'].isnull()])

492

In [233]:
len(merged[merged['ARV'].isnull()])

689

In [234]:
merged.loc[merged['ARV'].isnull(), 'ARV'] = merged.loc[merged['ARV'].isnull(), 'estARV']
len(merged[merged['ARV'].isnull()])

581

In [235]:
merged.loc[merged['ARV'].isnull(), 'ARV'] = merged.loc[merged['ARV'].isnull(), 'Text23']
len(merged[merged['ARV'].isnull()])

554

In [236]:
merged.loc[merged['ARV'].isnull(), 'ARV'] = merged.loc[merged['ARV'].isnull(), 'After Repaired Value (ARV)']
len(merged[merged['ARV'].isnull()])

541

In [237]:
merged = merged[merged['Loan Amount'].notnull()]
merged[merged['Loan Amount'].isnull()]

,90DO,ALTA,ARV,ARVP,Application Date,B1N,Borrower,Borrower Acknowledgment,CD,CLOF,...,What is your investment strategy for the property?,What is your marital status?,What is your strategy for the investment property?,What type of investment property?,What will be the occupancy after closing?,Which of the following apply?,"Who is the Buyer (or Owner, if a refi) for the property?",Who needs notification for property access?,Who were you referred to?,Years at Job


In [238]:
len(merged[merged['Repeat Borrower'].isnull()])

694

In [239]:
len(merged[merged['IE'].isnull()])

688

In [240]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'email']
len(merged[merged['IE'].isnull()])

579

In [241]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'Email_x']
len(merged[merged['IE'].isnull()])

436

In [242]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'Email_y']
len(merged[merged['IE'].isnull()])

427

In [243]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'Email Address']
len(merged[merged['IE'].isnull()])

426

In [244]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'REemail']
len(merged[merged['IE'].isnull()])

426

In [245]:
merged.loc[merged['IE'].isnull(), 'IE'] = merged.loc[merged['IE'].isnull(), 'RE Email']
len(merged[merged['IE'].isnull()])

426

In [246]:
def is_repeat_client(x):
    is_repeat = '0'
    if '@' in str(x['temp']):
        is_repeat = '1'
    
    return is_repeat

merged['temp'] = ''
merged['temp'] = merged[merged['IE'].notnull() & merged.duplicated(['IE'])]['IE']
merged['Repeat Client'] = merged.apply(is_repeat_client, axis = 1)


In [247]:
merged['Repeat Client'][:5]

0    0
1    0
2    0
3    0
4    0
Name: Repeat Client, dtype: object

In [248]:
len(merged[merged['Repeat Borrower'].isnull()])

694

In [249]:
merged.loc[merged['Repeat Borrower'].isnull(), 'Repeat Borrower'] = merged.loc[merged['Repeat Borrower'].isnull(), 'Repeat Client']
len(merged[merged['Repeat Borrower'].isnull()])

0

In [250]:
len(merged[merged['Years of Experience'].isnull()])

694

In [251]:
merged.loc[merged['Years of Experience'].isnull(), 'Years of Experience'] = merged.loc[merged['Years of Experience'].isnull(), 'Text54']
len(merged[merged['Years of Experience'].isnull()])

665

In [252]:
merged.loc[merged['Years of Experience'].isnull(), 'Years of Experience'] = merged.loc[merged['Years of Experience'].isnull(), 'yearsOfExp']
len(merged[merged['Years of Experience'].isnull()])

565

In [253]:
merged.loc[merged['Years of Experience'].isnull(), 'Years of Experience'] = merged.loc[merged['Years of Experience'].isnull(), 'How many years of experience?']
len(merged[merged['Years of Experience'].isnull()])

551

In [254]:
len(merged[merged['Completed Properties'].isnull()])

694

In [255]:
merged.loc[merged['Completed Properties'].isnull(), 'Completed Properties'] = merged.loc[merged['Completed Properties'].isnull(), 'numInvProps']
len(merged[merged['Completed Properties'].isnull()])

595

In [256]:
merged['Text57'] = merged['Text57'].astype(float).fillna(0.0)
merged['Text58'] = merged[merged['Text58'].notnull() & merged['Text58'].str.isdigit()]['Text58'].astype(float).fillna(0.0)

In [257]:
merged['Temp2'] = merged['Text57'] + merged['Text58']

In [258]:
merged.loc[merged['Completed Properties'].isnull(), 'Completed Properties'] = merged.loc[merged['Completed Properties'].isnull(), 'Temp2']
len(merged[merged['Completed Properties'].isnull()])

586

In [259]:
merged.loc[merged['Completed Properties'].isnull(), 'Completed Properties'] = merged.loc[merged['Completed Properties'].isnull(), 'Properties Completed?']
len(merged[merged['Completed Properties'].isnull()])

570

In [260]:
len(merged[merged['Repair Credit Line'].isnull()])

232

In [261]:
merged.loc[merged['Repair Credit Line'].isnull(), 'Repair Credit Line'] = merged.loc[merged['Repair Credit Line'].isnull(), 'RCL Amt']
len(merged[merged['Repair Credit Line'].isnull()])

228

In [262]:
merged.loc[merged['Repair Credit Line'].isnull(), 'Repair Credit Line'] = merged.loc[merged['Repair Credit Line'].isnull(), 'RE']
len(merged[merged['Repair Credit Line'].isnull()])

215

In [263]:
merged.loc[merged['Repair Credit Line'].isnull(), 'Repair Credit Line'] = merged.loc[merged['Repair Credit Line'].isnull(), 'estRehab']
len(merged[merged['Repair Credit Line'].isnull()])

212

In [264]:
merged.loc[merged['Repair Credit Line'].isnull(), 'Repair Credit Line'] = merged.loc[merged['Repair Credit Line'].isnull(), 'Text22']
len(merged[merged['Repair Credit Line'].isnull()])

212

In [265]:
merged.loc[merged['Repair Credit Line'].isnull(), 'Repair Credit Line'] = merged.loc[merged['Repair Credit Line'].isnull(), 'Rehab']
len(merged[merged['Repair Credit Line'].isnull()])

209

In [266]:
len(merged[merged['Closing Date'].isnull()])

161

In [267]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'CD']
len(merged[merged['Closing Date'].isnull()])

83

In [268]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Est Closing Date']
len(merged[merged['Closing Date'].isnull()])

69

In [269]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'estClosingDate']
len(merged[merged['Closing Date'].isnull()])

67

In [270]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Estimated closing date?']
len(merged[merged['Closing Date'].isnull()])

65

In [271]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Date_x']
len(merged[merged['Closing Date'].isnull()])

64

In [272]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Date_y']
len(merged[merged['Closing Date'].isnull()])

64

In [273]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'DateSigned1']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'DateSigned2']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'TodaysDate']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'datePropPurchased']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Date']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Application Date']
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Application Date']
merged['Closing Date'].replace('TBD', '')
merged['Closing Date'].replace('ASAP', '')
#merged[merged['Closing Date'].notnull() and merged['Closing Date'].str.contains('TBD')] = ''
len(merged[merged['Closing Date'].isnull()])

7

In [274]:
property_features = pd.read_csv(open('property_features.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)
len(property_features)

1160

In [275]:
merged = pd.merge(merged, property_features, how = 'left', on = 'TransactionID')
len(merged)

1161

In [276]:
purchase_price = pd.read_csv(open('all_transactions_PP.csv','rU'), encoding = 'utf-8', engine = 'c', error_bad_lines = False)
purchase_price = purchase_price[['TransactionID', 'Purchase Price']]
len(purchase_price)

1160

In [277]:
purchase_price['TransactionID'] = purchase_price['TransactionID'].str.zfill(5)

In [278]:
merged= pd.merge(merged, purchase_price, how = 'left', on = 'TransactionID')
len(merged)

1161

In [279]:
merged.loc[merged['PP'].isnull(), 'PP'] = merged.loc[merged['Purchase Price'].notnull(), 'Purchase Price']
merged[merged['PP'].isnull()][['PP', 'TransactionID']]

,PP,TransactionID
29,NaN,5507S
54,NaN,1210N
319,NaN,4827C
422,NaN,6910C
442,NaN,6082S
649,NaN,04452
791,NaN,4365N
985,NaN,BlanL


In [280]:
#len(merged[merged['approxHeatedSf'].notnull()]['approxHeatedSf'])

In [281]:
#merged.loc[merged['approxHeatedSf'].isnull(), 'approxHeatedSf'] = merged.loc[merged['approxHeatedSf'].isnull(), 'Text19']
#merged.loc[merged['approxHeatedSf'].isnull(), 'approxHeatedSf'] = merged.loc[merged['approxHeatedSf'].isnull(), 'Approx. Heated Sf']


In [282]:
#len(merged[merged['approxHeatedSf'].notnull()]['approxHeatedSf'])

In [283]:
#len(merged[merged['beds'].notnull()]['beds'])

In [284]:
#len(merged[merged['bath'].notnull()]['bath'])

In [285]:
#merged.loc[merged['beds'].isnull(), 'beds'] = merged.loc[merged['beds'].isnull(), 'Text17']
#merged.loc[merged['beds'].isnull(), 'beds'] = merged.loc[merged['beds'].isnull(), 'Beds']

#merged.loc[merged['bath'].isnull(), 'bath'] = merged.loc[merged['bath'].isnull(), 'Text18']
#merged.loc[merged['bath'].isnull(), 'bath'] = merged.loc[merged['bath'].isnull(), 'Baths']


In [286]:
#len(merged[merged['beds'].notnull()]['beds'])

In [287]:
#len(merged[merged['bath'].notnull()]['bath'])

In [288]:
import datetime

def parse_date(x):  
    try:
        x['Closing Date'] = datetime.datetime.strptime(x['Closing Date'], "%m/%d/%y").strftime("%Y-%m")
        return x['Closing Date'] 
    except:
        try:
            x['Closing Date'] = datetime.datetime.strptime(x['Closing Date'], "%m/%d/%Y").strftime("%Y-%m")
            return x['Closing Date']
        except:
            try:
                x['Closing Date'] = datetime.datetime.strptime(x['Closing Date'], "%d-%b").strftime(str(datetime.datetime.now().year)+"-%m")
                return x['Closing Date']
            except:
                try:
                    x['Closing Date'] = datetime.datetime.strptime(x['Closing Date'], "%Y-%m").strftime("%Y-%m")
                    return x['Closing Date']
                except:
                    return np.nan
    

merged['Closing Date'] = merged[~merged['Closing Date'].isnull()].apply(parse_date, axis = 1)
merged['Closing Date'][-5:]

1156    2015-11
1157    2016-12
1158    2016-02
1159    2016-02
1160    2016-02
Name: Closing Date, dtype: object

In [289]:
len(merged[merged['Closing Date'].isnull()])

25

In [290]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = merged.loc[merged['Closing Date'].isnull(), 'Application Date']
len(merged[merged['Closing Date'].isnull()])

9

In [291]:
merged['Closing Date'] = merged[~merged['Closing Date'].isnull()].apply(parse_date, axis = 1)
merged['Closing Date'][-5:]

1156    2015-11
1157    2016-12
1158    2016-02
1159    2016-02
1160    2016-02
Name: Closing Date, dtype: object

In [292]:
len(merged[merged['Closing Date'].isnull()])

9

In [293]:
merged.loc[merged['Closing Date'].isnull(), 'Closing Date'] = '2016-1'
merged.loc[merged['RegionName'].isnull(), 'RegionName'] = '32805'

In [294]:
len(merged[merged['Closing Date'].isnull()])

0

In [295]:
merged['Closing Date'][:5]

0    2011-09
1    2011-09
2    2011-09
3    2011-09
4    2011-09
Name: Closing Date, dtype: object

In [296]:
def find_quarter(x):
    month = datetime.datetime.strptime(x['Closing Date'], "%Y-%m").strftime("%m")
    return (math.ceil(int(month)/3.0))

merged['Quarter'] = merged[merged['Closing Date'].notnull()].apply(find_quarter, axis = 1)
merged[['Quarter', 'Closing Date']][:10]

,Quarter,Closing Date
0,3,2011-09
1,3,2011-09
2,3,2011-09
3,3,2011-09
4,3,2011-09
5,3,2011-09
6,3,2011-09
7,3,2011-09
8,3,2011-09
9,4,2011-10


In [297]:
len(merged[merged['RegionName'].isnull()])

0

In [298]:
merged.to_csv('all_transactions.csv')

In [299]:
len(merged.columns)

511

In [300]:
#merged = merged.dropna(thresh=len(merged) - 650, axis='columns')
#len(merged.columns)

In [301]:
#merged.to_csv('all_transactions.csv')

In [302]:
#home_values = pd.read_csv(urllib2.urlopen('http://files.zillowstatic.com/research/public/Zip/Zip_MedianSoldPricePerSqft_AllHomes.csv'))
home_values = pd.read_csv(open('Zillow_Data/Zip_MedianSoldPricePerSqft_AllHomes.csv','rU'), encoding = 'ISO-8859-1', engine = 'c', error_bad_lines = False)#, index_col='RegionName')
home_values = home_values.drop('RegionID', 1)
home_values = home_values.drop('City', 1)
home_values = home_values.drop('State', 1)
home_values = home_values.drop('Metro', 1)
home_values = home_values.drop('CountyName', 1)
home_values = home_values.drop('SizeRank', 1)
home_values[:5]

,RegionName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01
0,11368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60629,83.3729,81.9039,86.2227,88.2281,92.4941,88.4356,84.2880,86.4587,87.2849,...,109.5701,116.6246,111.9092,114.7804,127.931300,132.5965,132.6476,130.141800,136.5036,133.4347
2,90650,117.3969,118.0225,113.3146,112.0347,114.1521,113.2268,111.2448,109.2356,112.6931,...,309.2683,316.2006,339.0004,341.2761,339.182500,336.7075,347.7496,337.655200,330.7219,331.5490
3,11226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,351.941748,NaN,NaN,408.496732,NaN,NaN


In [303]:
median_sales_price = pd.read_csv(open('Zillow_Data/Zip_Zhvi_SingleFamilyResidence.csv','rU'), encoding = 'ISO-8859-1', engine = 'c', error_bad_lines = False)#, index_col='RegionName')
median_sales_price = median_sales_price.drop('RegionID', 1)
median_sales_price = median_sales_price.drop('City', 1)
median_sales_price = median_sales_price.drop('State', 1)
median_sales_price = median_sales_price.drop('Metro', 1)
median_sales_price = median_sales_price.drop('CountyName', 1)
median_sales_price = median_sales_price.drop('SizeRank', 1)
median_sales_price[:5]

,RegionName,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01
0,11368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,559800,567300,573800,578200,584300,593100,602100,611900,624100,634800
1,79936,70900,71200,71100,71300,71400,71900,72700,73100,72700,...,111400,111400,111500,111900,112000,111600,111200,111000,110700,110500
2,60629,93600,93600,93400,93300,93000,92700,92500,92500,92400,...,136200,137100,138900,141000,142100,141800,140800,140300,140400,140600
3,90650,145600,145300,144700,144100,143600,143200,142700,142300,141900,...,386100,388900,391300,393600,396500,399400,401500,403400,405900,408200
4,11226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,797600,806100,811100,817000,824800,821700,807200,805800,822700,841100


In [304]:
columns = list(home_values.columns.values)
def findColumns(x):
    try:
        #return home_values[home_values['RegionName']==x.astype(str)[:5]]
        return (columns.index(x['Closing Date']))
        #return '1'
    except ValueError, TypeError:
        return (columns.index('2016-01'))

f = lambda x: findColumns(x)

#Changed application date of last three rows to be within the range of zillows home value data, row 1176, 1180, 1187 and 1188
cols = merged[~merged['Closing Date'].isnull()].apply(f, axis = 1)
columns = list(median_sales_price.columns.values)
cols2 = merged[~merged['Closing Date'].isnull()].apply(f, axis = 1)

len(cols)

1161

In [305]:
from pyzipcode import ZipCodeDatabase
#rows = list(home_values.index.values)
rows = list(home_values['RegionName'])
def findZip(x, rows):
    try:
        
        #return home_values[home_values['RegionName']==x.astype(str)[:5]]
        return (rows.index(int(x)))
    except ValueError, TypeError:
        zcdb = ZipCodeDatabase()
        try:
            zipcodes = [z.zip for z in zcdb.get_zipcodes_around_radius(x, 2)]
            for zipcode in zipcodes:
                try:
                    rowsindex = rows.index(int(zipcode))
                    print zipcode + ',' + rowsindex
                    return rowsindex
                except:
                    continue
            return rows.index(32804)
        except:
            return rows.index(32804)

f = lambda x: findZip(x, rows)
rowIndeces = merged[~merged['RegionName'].isnull()]['RegionName'].map(f)
rows = list(median_sales_price['RegionName'])
rowIndeces2 = merged[~merged['RegionName'].isnull()]['RegionName'].map(f)

len(rowIndeces)

1161

In [306]:
merged['MedianSalesPriceSqFt'] = zip(rowIndeces, cols)
merged['MedianSalesPrice'] = zip(rowIndeces2, cols2)

#f = lambda x: home_values.iloc[rowIndeces, columnIndeces]
f = lambda x: home_values.iloc[(int(x[0]))][(int(x[1]))]
merged['MedianSalesPriceSqFt'] = merged['MedianSalesPriceSqFt'].map(f)
f = lambda x: median_sales_price.iloc[(int(x[0]))][(int(x[1]))]
merged['MedianSalesPrice'] = merged['MedianSalesPrice'].map(f)

len(merged[merged['MedianSalesPrice'].isnull()])

0

In [307]:
#home_values = pd.read_csv(urllib2.urlopen('http://files.zillowstatic.com/research/public/Zip/Zip_MedianSoldPricePerSqft_AllHomes.csv'))
market_health = pd.read_csv(open('Zillow_Data/MarketHealthIndex_Zip.csv','rU'), encoding = 'ISO-8859-1', engine = 'c', error_bad_lines = False)#, index_col='RegionName')
market_health = market_health.drop('RegionType', 1)
market_health = market_health.drop('City', 1)
market_health = market_health.drop('State', 1)
market_health = market_health.drop('Metro', 1)
market_health = market_health.drop('CBSATitle', 1)
market_health = market_health.drop('SizeRank', 1)
market_health[:5]

,RegionName,MarketHealthIndex,SellForGain,PrevForeclosed,ForeclosureRatio,ZHVI,MoM,YoY,ForecastYoYPctChange,StockOfREOs,NegativeEquity,Delinquency,DaysOnMarket
0,1001,3.883605,100,NaN,4.3499,183200,0.054615,2.346369,0.019045,22.2,0.128492,0.063768,112.5
1,1002,2.949879,60,NaN,6.3783,301700,0.033156,0.499667,0.013271,14.8,0.058672,0.014599,101.5
2,1005,2.875906,NaN,NaN,NaN,193300,0.415584,4.430038,0.028614,32.3,0.150367,0.105691,103.0
3,1007,2.950634,75,NaN,NaN,252700,0.198255,1.282565,0.018951,16.1,0.053743,0.064286,101.5
4,1008,4.730525,100,NaN,46.5116,222700,0.632625,1.735952,0.019520,17.5,0.067073,0.090909,112.5


In [308]:
rows = list(market_health['RegionName'])
columns = list(market_health.columns.values)
columns

[u'RegionName',
 u'MarketHealthIndex',
 u'SellForGain',
 u'PrevForeclosed',
 u'ForeclosureRatio',
 u'ZHVI',
 u'MoM',
 u'YoY',
 u'ForecastYoYPctChange',
 u'StockOfREOs',
 u'NegativeEquity',
 u'Delinquency',
 u'DaysOnMarket']

In [309]:

def get_market_data(x, marketdata):
    x = float(x)
    try:
        rowIndex = rows.index(x)
        columnIndex = columns.index(marketdata)
        
        return market_health.iloc[(rowIndex)][(columnIndex)]
    except:
        return np.nan

f = lambda x: get_market_data(str(x), 'MarketHealthIndex')


merged['MarketHealthIndex'] = merged[~merged['RegionName'].isnull()]['RegionName'].map(f)
merged['MarketHealthIndex'][:5]

0    2.181461
1    0.399306
2    0.935236
3    1.668176
4    2.292421
Name: MarketHealthIndex, dtype: float64

In [310]:
f = lambda x: get_market_data(str(x), 'YoY')


merged['YoY'] = merged[~merged['RegionName'].isnull()]['RegionName'].map(f)
merged['YoY'][:5]

0    6.772674
1   -0.835655
2    5.844156
3    9.523810
4    6.383750
Name: YoY, dtype: float64

In [311]:
f = lambda x: get_market_data(str(x), 'DaysOnMarket')


merged['DaysOnMarket'] = merged[~merged['RegionName'].isnull()]['RegionName'].map(f)
merged['DaysOnMarket'][:5]

0     84.5
1    102.5
2     84.0
3    107.5
4     95.0
Name: DaysOnMarket, dtype: float64

In [312]:
from pyzipcode import ZipCodeDatabase
import math
def find_data_nearby_zips(x, marketdata):
    x = str(int(x))
    zcdb = ZipCodeDatabase()
    try:
        zipcodes = [z.zip for z in zcdb.get_zipcodes_around_radius(x, 10)]
        for zipcode in zipcodes:
            market_data = get_market_data(zipcode, marketdata)
            if market_data is not np.nan:
                #print zipcode + ',' + x
                return market_data
        return get_maret_data('32804', marketdata)
    except:
        #print '32804'
        return get_market_data('32804', marketdata)

f = lambda x: find_data_nearby_zips(x, 'MarketHealthIndex')
merged.loc[merged['MarketHealthIndex'].isnull(),'MarketHealthIndex'] = merged.loc[merged['MarketHealthIndex'].isnull(),'RegionName'].map(f)
f = lambda x: find_data_nearby_zips(x, 'YoY')
merged.loc[merged['YoY'].isnull(),'YoY'] = merged.loc[merged['YoY'].isnull(),'RegionName'].map(f)
f = lambda x: find_data_nearby_zips(x, 'DaysOnMarket')
merged.loc[merged['DaysOnMarket'].isnull(),'DaysOnMarket'] = merged.loc[merged['DaysOnMarket'].isnull(),'RegionName'].map(f)

In [313]:
print len(merged[merged['MarketHealthIndex'].isnull()]['MarketHealthIndex'])
print len(merged[merged['YoY'].isnull()]['YoY'])
print len(merged[merged['DaysOnMarket'].isnull()]['DaysOnMarket'])

0
0
3


In [314]:
dataset = merged[['TransactionID', 'Loan Amount', 'Closing Date', 'Quarter', 'Closed', 'RegionName', 'PP', 'ARV', 'Beds_y', 'Bath', 'SqFt', 'Repair Credit Line', 'cashReserves', 'Repeat Borrower', 'Years of Experience', 'Completed Properties', 'MarketHealthIndex', 'YoY', 'DaysOnMarket', 'MedianSalesPriceSqFt', 'MedianSalesPrice']]


In [315]:
len(dataset[dataset['PP'].notnull()])

1153

In [316]:
#def clean_dollar_amounts(column):
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace('$','')
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace(',','')
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace('k','000')
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace('K','000')
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace('>','')
    #dataset.loc[dataset[column].notnull(), column] = dataset[dataset[column].notnull()][column].str.replace('+','')
    
    #dataset[column].convert_objects(convert_numeric = True)
    #dataset.loc[dataset[column].notnull(), column] =  = int(dataset[dataset[column].notnull()][column])


#columns = ['Repair Credit Line', 'cashReserves', 'Loan Amount', 'ARV', 'PP']

#for column in columns:
#    clean_dollar_amounts(column);

#dataset[-5:]

In [317]:
def extract_numbers(x):
    x = str(x)
    x = x.replace('.0','')
    x = x.replace('.00','')
    x = x.replace('k','000')
    x = x.replace('K','000')
    value = re.sub(r"[^0-9\.]+", "", str(x))
    return value

columns = ['Repair Credit Line', 'cashReserves', 'Loan Amount', 'ARV', 'PP', 'Beds_y', 'Bath', 'SqFt']
#dataset['PP'] = dataset[~dataset['PP'].isnull()]['PP'].map(extract_numbers)
#dataset.loc[dataset['cashReserves'].notnull(), 'cashReserves'] = dataset.loc[dataset['cashReserves'].notnull(), 'cashReserves'].map(extract_numbers)

for column in columns:
    dataset.loc[dataset[column].notnull(), column] = dataset.loc[dataset[column].notnull(), column].map(extract_numbers)



In [318]:
len(dataset[dataset['PP'].isnull()])

8

In [319]:
dataset.columns = ['TransactionID', 'LoanAmount', 'ClosingDate', 'Quarter', 'Closed', 'RegionName', 'PurchasePrice', 'ARV', 'Beds', 'Bath', 'SqFt', 'RepairCreditLine', 'CashReserves', 'RepeatBorrower', 'YearsOfExperience', 'CompletedProperties', 'MarketHealthIndex', 'YoY', 'DaysOnMarket', 'MedianSalesPriceSqFt', 'MedianSalesPrice']

In [320]:
dataset.to_csv('all_transactions.csv')